In [1]:
import keras
from keras.layers import (Dense, Dropout, Conv2D, GlobalAveragePooling2D, MaxPool2D, Flatten, Activation, Input, BatchNormalization)
from keras.preprocessing. image import  ImageDataGenerator
from keras.models import  Model
from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau
from keras.optimizers import Adam, RMSprop
import numpy as np
import pandas as pd
from keras.applications.vgg19 import VGG19

from load_test_seedling import load_test_data
from load_seedling_data import load_data

Using TensorFlow backend.


In [2]:
BATCH_SIZE = 64
SZ = 299 # Image height and width
data_dir = '/home/tushar/Datasets/seedlingDataset/train'

In [3]:
# Loading pretrained VGG19 model on imagenet
base_model = VGG19(include_top=False, weights='imagenet', input_shape=(SZ,SZ,3))

In [4]:
# Adding Global Average pooling layer and some dense layers and dropout with softmax output
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.6)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
prediction = Dense(12, activation='softmax')(x)

In [5]:
# Making model object for the new model
model = Model(inputs = base_model.input, outputs = prediction)

In [6]:
# So only added layers are trainable
for l in base_model.layers:
    l.trainable = False

In [7]:
# optimizer
opt = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

In [8]:
# Compiling model
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
# Loading the data using helper functions with 80 20 split
x_train, y_train, x_valid, y_valid, class_dict, class_dict_inv = load_data(data_dir=data_dir,
                                                                           sz=SZ,
                                                                           train_split=.80,
                                                                           rescale=1.255)


  0%|          | 0/12 [00:00<?, ?it/s]

4750 images found


100%|██████████| 12/12 [00:43<00:00,  3.63s/it]


In [10]:
# Training for 5 epochs
info  = model.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=10, validation_data=(x_valid, y_valid))

Train on 3800 samples, validate on 950 samples
Epoch 1/10
3800/3800 [==============================] - 36s 9ms/step - loss: 3.9897 - acc: 0.2061 - val_loss: 2.0007 - val_acc: 0.4126
Epoch 2/10
3800/3800 [==============================] - 29s 8ms/step - loss: 2.1798 - acc: 0.3047 - val_loss: 1.8668 - val_acc: 0.4579
Epoch 3/10
3800/3800 [==============================] - 29s 8ms/step - loss: 1.8787 - acc: 0.3839 - val_loss: 1.5948 - val_acc: 0.5516
Epoch 4/10
3800/3800 [==============================] - 30s 8ms/step - loss: 1.7033 - acc: 0.4258 - val_loss: 1.3561 - val_acc: 0.6316
Epoch 5/10
3800/3800 [==============================] - 30s 8ms/step - loss: 1.5213 - acc: 0.4861 - val_loss: 1.1923 - val_acc: 0.6579
Epoch 6/10
3800/3800 [==============================] - 29s 8ms/step - loss: 1.4354 - acc: 0.5079 - val_loss: 1.0659 - val_acc: 0.6916
Epoch 7/10
3800/3800 [==============================] - 29s 8ms/step - loss: 1.3381 - acc: 0.5358 - val_loss: 0.9737 - val_acc: 0.7211
Epoch 8/

In [11]:
model.save_weights("temp_vgg19")


In [12]:
model.load_weights("temp_vgg19")

In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 299, 299, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 299, 299, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 299, 299, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 149, 149, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 149, 149, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 149, 149, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 74, 74, 128)       0         
__________

In [14]:
# Finetuning the model
for l in model.layers[-10:]:
    l.trainable = True
# Setting a smaller learning rate so that weights are not changed a lot and using reducing lr on plateau and saving using checkpoint
opt = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose = 1, mode = 'auto', )
msave = ModelCheckpoint('vgg19_model_checkpoint.hdf5', save_best_only=True)
model.compile(opt, 'categorical_crossentropy', ['accuracy'])

In [15]:
# Training the model for 20 epochs
info  = model.fit(x_train, 
                  y_train, 
                  batch_size=BATCH_SIZE, 
                  epochs=50, 
                  validation_data=(x_valid, y_valid), 
                  callbacks=[lr_reduce, msave])

Train on 3800 samples, validate on 950 samples
Epoch 1/50
3800/3800 [==============================] - 30s 8ms/step - loss: 0.9760 - acc: 0.6695 - val_loss: 0.4320 - val_acc: 0.8600
Epoch 2/50
3800/3800 [==============================] - 30s 8ms/step - loss: 0.5462 - acc: 0.8153 - val_loss: 0.4314 - val_acc: 0.8389
Epoch 3/50
3800/3800 [==============================] - 31s 8ms/step - loss: 0.3932 - acc: 0.8624 - val_loss: 0.3699 - val_acc: 0.8695
Epoch 4/50
3800/3800 [==============================] - 29s 8ms/step - loss: 0.3336 - acc: 0.8847 - val_loss: 0.2702 - val_acc: 0.8958
Epoch 5/50
3800/3800 [==============================] - 29s 8ms/step - loss: 0.2411 - acc: 0.9139 - val_loss: 0.2494 - val_acc: 0.9221
Epoch 6/50
3800/3800 [==============================] - 29s 8ms/step - loss: 0.1958 - acc: 0.9318 - val_loss: 0.2666 - val_acc: 0.9084
Epoch 7/50
3800/3800 [==============================] - 29s 8ms/step - loss: 0.1700 - acc: 0.9392 - val_loss: 0.2447 - val_acc: 0.9179
Epoch 8/

<h1>Future Upgrades</h1>
1. Data Augmentation during training
2. l2 Regularization
3. Batch Norm
